# Circuit Basics - Code Laboratory

**Section 3: Create Circuits - Part 1** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

### Circuit Creation
| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `QuantumCircuit()` | `QuantumCircuit(n_qubits, n_clbits=0)` | `QuantumCircuit` | Simple circuit creation |
| `QuantumCircuit()` | `QuantumCircuit(qr, cr)` | `QuantumCircuit` | Using named registers |
| `QuantumRegister()` | `QuantumRegister(size, name=None)` | `QuantumRegister` | Named quantum register |
| `ClassicalRegister()` | `ClassicalRegister(size, name=None)` | `ClassicalRegister` | Named classical register |

### Circuit Properties (⚠️ EXAM CRITICAL!)
| Attribute/Method | Type | Returns | Description |
|------------------|------|---------|-------------|
| `qc.num_qubits` | **Property** | `int` | Number of qubits (NO parentheses!) |
| `qc.num_clbits` | **Property** | `int` | Number of classical bits (NO parentheses!) |
| `qc.depth()` | **Method** | `int` | Critical path length (WITH parentheses!) |
| `qc.size()` | **Method** | `int` | Total gate count (WITH parentheses!) |
| `qc.width()` | **Method** | `int` | Total wires: qubits + clbits (WITH parentheses!) |
| `qc.count_ops()` | **Method** | `dict` | Dictionary of {gate: count} |

---

In [ ]:
"""
Qiskit Code Laboratory - Circuit Basics
========================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, Operator

# Utility functions for this notebook
def verify_state(qc, expected_state, tolerance=1e-10):
    """Verify circuit produces expected state."""
    actual = Statevector.from_instruction(qc)
    return np.allclose(actual.data, expected_state, atol=tolerance)

def show_state(qc, label=""):
    """Display statevector with optional label."""
    sv = Statevector.from_instruction(qc)
    print(f"{label}: {sv}" if label else f"{sv}")
    return sv

def verify_property(value, expected, name):
    """Verify a circuit property matches expected value."""
    if value == expected:
        print(f"✅ {name} = {value}")
        return True
    else:
        print(f"❌ {name} = {value} (expected {expected})")
        return False

print("✅ Environment ready - Circuit Basics Lab")

---

## `QuantumCircuit()` - Circuit Creation

### Signature
```python
QuantumCircuit(n_qubits: int, n_clbits: int = 0) -> QuantumCircuit
QuantumCircuit(*regs: QuantumRegister | ClassicalRegister) -> QuantumCircuit
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `n_qubits` | `int` | Yes | - | Number of quantum bits |
| `n_clbits` | `int` | No | `0` | Number of classical bits |
| `*regs` | `Register` | Alt | - | Variable quantum/classical registers |

### Returns
`QuantumCircuit` - The created circuit object

---

### Basic Usage: Simple Creation

In [ ]:
# ============================================================
# QuantumCircuit() - BASIC USAGE
# ============================================================

# Method 1: Just qubits (no classical bits needed yet)
qc1 = QuantumCircuit(2)
print("Circuit with 2 qubits:")
print(qc1.draw())

# Verify properties
assert qc1.num_qubits == 2, "Should have 2 qubits"
assert qc1.num_clbits == 0, "Should have 0 classical bits"
print("✅ Basic circuit verified")

print("\n" + "="*50)

# Method 2: Qubits + classical bits (for measurement)
qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.cx(0, 1)
qc2.measure([0, 1], [0, 1])

print("\nCircuit with 2 qubits and 2 classical bits:")
print(qc2.draw())

# Verify properties
assert qc2.num_qubits == 2, "Should have 2 qubits"
assert qc2.num_clbits == 2, "Should have 2 classical bits"
print("✅ Circuit with measurements verified")

---

### Parameter Variations: Named Registers

In [ ]:
# ============================================================
# QuantumCircuit() - PARAMETER VARIATIONS: Named Registers
# ============================================================

# Create named registers
qr = QuantumRegister(3, 'q')   # 3 qubits named 'q'
cr = ClassicalRegister(3, 'c')  # 3 classical bits named 'c'

qc = QuantumCircuit(qr, cr)
qc.h(qr[0])
qc.cx(qr[0], qr[1])
qc.cx(qr[1], qr[2])
qc.measure(qr, cr)

print("Circuit with named registers:")
print(qc.draw())

# Verify register names
assert qr.name == 'q', "Quantum register should be named 'q'"
assert cr.name == 'c', "Classical register should be named 'c'"
assert qc.num_qubits == 3, "Should have 3 qubits"
assert qc.num_clbits == 3, "Should have 3 classical bits"
print("✅ Named registers verified")

---

### Advanced: Multiple Registers

In [ ]:
# ============================================================
# QuantumCircuit() - ADVANCED: Multiple Registers
# ============================================================

# Separate logical units
data_qubits = QuantumRegister(4, 'data')
ancilla_qubits = QuantumRegister(2, 'ancilla')
measurements = ClassicalRegister(4, 'meas')
syndrome = ClassicalRegister(2, 'syndrome')

qc = QuantumCircuit(data_qubits, ancilla_qubits, measurements, syndrome)

# Operations on specific registers
qc.h(data_qubits)  # Hadamard on all data qubits
qc.x(ancilla_qubits[0])  # X on first ancilla

# Measure to specific registers
qc.measure(data_qubits, measurements)
qc.measure(ancilla_qubits, syndrome)

print("Circuit with multiple registers:")
print(qc.draw())

# Verify total counts
assert qc.num_qubits == 6, f"Expected 6 qubits (4 data + 2 ancilla), got {qc.num_qubits}"
assert qc.num_clbits == 6, f"Expected 6 clbits (4 meas + 2 syndrome), got {qc.num_clbits}"
print("✅ Multiple registers verified")

---

## Circuit Properties - `depth()`, `size()`, `width()`

### Signatures
```python
QuantumCircuit.depth() -> int      # Critical path length
QuantumCircuit.size() -> int       # Total gate count  
QuantumCircuit.width() -> int      # Total wires (qubits + clbits)
QuantumCircuit.count_ops() -> dict # Operation breakdown
```

### Properties (NO parentheses!)
```python
QuantumCircuit.num_qubits -> int   # PROPERTY - no ()
QuantumCircuit.num_clbits -> int   # PROPERTY - no ()
```

---

### Basic Usage: The Big Three

In [ ]:
# ============================================================
# Circuit Properties - BASIC USAGE
# ============================================================

# Create a GHZ state circuit
qc = QuantumCircuit(3, 3)
qc.h(0)              # Layer 1
qc.cx(0, 1)          # Layer 2
qc.cx(1, 2)          # Layer 3
qc.measure([0,1,2], [0,1,2])  # Layer 4

print("GHZ State Circuit:")
print(qc.draw())

print("\n" + "="*50)
print("CIRCUIT PROPERTIES")
print("="*50)

# METHODS (with parentheses!)
depth = qc.depth()
size = qc.size()
width = qc.width()
ops = qc.count_ops()

# PROPERTIES (no parentheses!)
n_qubits = qc.num_qubits
n_clbits = qc.num_clbits

# Display and verify
verify_property(depth, 4, "depth()")
print("   → Layers: H → CX(0,1) → CX(1,2) → Measure")

verify_property(size, 6, "size()")
print("   → Count: 1 H + 2 CX + 3 Measure = 6")

verify_property(width, 6, "width()")
print("   → Count: 3 qubits + 3 classical = 6 wires")

verify_property(n_qubits, 3, "num_qubits")
print("   → ⚠️ PROPERTY (no parentheses!)")

verify_property(n_clbits, 3, "num_clbits")
print("   → ⚠️ PROPERTY (no parentheses!)")

print(f"\ncount_ops() = {ops}")
print("   → Dictionary of gate types and counts")

---

### ⚠️ TRAP DEMONSTRATION: Property vs Method

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Property vs Method
# This is what the exam tests CONSTANTLY!
# ============================================================

print("⚠️ TRAP: Property vs Method")
print("="*50)

qc = QuantumCircuit(3, 2)
qc.h([0, 1, 2])

# ✅ CORRECT: Properties (no parentheses)
print("\n✅ CORRECT - Properties (no parentheses):")
print(f"   qc.num_qubits = {qc.num_qubits}")
print(f"   qc.num_clbits = {qc.num_clbits}")

# ✅ CORRECT: Methods (with parentheses)  
print("\n✅ CORRECT - Methods (with parentheses):")
print(f"   qc.depth() = {qc.depth()}")
print(f"   qc.size() = {qc.size()}")
print(f"   qc.width() = {qc.width()}")

# ❌ WRONG: Calling property as method
print("\n❌ WRONG - Calling property as method:")
try:
    result = qc.num_qubits()  # This FAILS!
    print(f"   Result: {result}")
except TypeError as e:
    print(f"   TypeError: {type(e).__name__}")
    print(f"   Message: 'int' object is not callable")

# ❌ WRONG: Calling method as property  
print("\n❌ WRONG - Calling method as property:")
method_ref = qc.depth  # This returns a METHOD REFERENCE, not the value!
print(f"   qc.depth = {method_ref}")
print(f"   Type: {type(method_ref)}")
print(f"   ↑ NOT the depth value! You got a bound method object!")

print("\n" + "="*50)
print("💡 RULE: 'num_*' = PROPERTY | everything else = METHOD()")
print("="*50)

---

## Depth Calculation - Parallel vs Sequential

### Key Concept
- **Parallel gates** → Same layer → Don't increase depth
- **Sequential gates** → Different layers → Increase depth

---

### Comparison: Parallel vs Sequential

In [ ]:
# ============================================================
# COMPARISON: Parallel vs Sequential Operations
# ============================================================

print("="*60)
print("COMPARING: Parallel vs Sequential")
print("="*60)

# Example 1: PARALLEL - All gates in same layer
qc_parallel = QuantumCircuit(3)
qc_parallel.h(0)  # Layer 1
qc_parallel.h(1)  # Layer 1 (parallel!)
qc_parallel.h(2)  # Layer 1 (parallel!)

print("\n📊 PARALLEL Hadamards (different qubits):")
print(qc_parallel.draw())
print(f"\nDepth: {qc_parallel.depth()}  ← Only 1! (all gates same layer)")
print(f"Size:  {qc_parallel.size()}   ← 3 gates total")

# Verify parallel behavior
assert qc_parallel.depth() == 1, "Parallel gates should have depth 1"
assert qc_parallel.size() == 3, "Should have 3 gates"
print("✅ Parallel behavior verified")

In [ ]:
# ============================================================
# SEQUENTIAL Operations (same qubit)
# ============================================================

qc_sequential = QuantumCircuit(1)
qc_sequential.h(0)  # Layer 1
qc_sequential.x(0)  # Layer 2
qc_sequential.z(0)  # Layer 3

print("📊 SEQUENTIAL Gates (same qubit):")
print(qc_sequential.draw())
print(f"\nDepth: {qc_sequential.depth()}  ← 3! (each gate different layer)")
print(f"Size:  {qc_sequential.size()}   ← 3 gates total")

# Verify sequential behavior
assert qc_sequential.depth() == 3, "Sequential gates should have depth 3"
assert qc_sequential.size() == 3, "Should have 3 gates"
print("✅ Sequential behavior verified")

In [ ]:
# ============================================================
# MIXED Operations (common exam question!)
# ============================================================

qc_mixed = QuantumCircuit(3)
qc_mixed.h(0)       # Layer 1 (q0)
qc_mixed.h(1)       # Layer 1 (q1) - parallel with H(0)
qc_mixed.cx(0, 2)   # Layer 2 (q0, q2) - depends on q0
qc_mixed.x(1)       # Layer 2 (q1) - parallel with CX, different qubits

print("📊 MIXED Parallel and Sequential:")
print(qc_mixed.draw())
print(f"\nDepth: {qc_mixed.depth()}")
print(f"Size:  {qc_mixed.size()}")

print("\n🔍 Layer Analysis:")
print("  Layer 1: H(0) and H(1) → parallel (different qubits)")
print("  Layer 2: CX(0,2) and X(1) → parallel (non-overlapping qubits)")

# Verify
assert qc_mixed.depth() == 2, "Mixed circuit should have depth 2"
assert qc_mixed.size() == 4, "Should have 4 gates"
print("✅ Mixed behavior verified")

---

### Verification Pattern: Depth Calculation Test Suite

In [ ]:
# ============================================================
# VERIFICATION PATTERN: Depth Calculation Test Suite
# ============================================================

def test_depth_calculation():
    """Test suite for understanding depth calculation."""
    
    # Test 1: All parallel gates
    qc1 = QuantumCircuit(4)
    qc1.h([0, 1, 2, 3])  # All H gates parallel
    assert qc1.depth() == 1, "Test 1 failed: 4 parallel H gates should have depth 1"
    
    # Test 2: Sequential on same qubit
    qc2 = QuantumCircuit(1)
    qc2.h(0)
    qc2.x(0)
    qc2.y(0)
    qc2.z(0)
    assert qc2.depth() == 4, "Test 2 failed: 4 sequential gates should have depth 4"
    
    # Test 3: Two-qubit gate dependency
    qc3 = QuantumCircuit(3)
    qc3.h(0)        # Layer 1
    qc3.h(1)        # Layer 1
    qc3.cx(0, 1)    # Layer 2 (needs q0 and q1)
    qc3.h(2)        # Layer 1 OR 2 (independent)
    assert qc3.depth() == 2, "Test 3 failed: Should be depth 2"
    
    # Test 4: Complex pattern
    qc4 = QuantumCircuit(4)
    qc4.h(0)        # Layer 1
    qc4.h(1)        # Layer 1
    qc4.h(2)        # Layer 1
    qc4.cx(0, 1)    # Layer 2
    qc4.cx(2, 3)    # Layer 2 (parallel - different qubits)
    qc4.x(0)        # Layer 3 (q0 busy in layer 2)
    assert qc4.depth() == 3, "Test 4 failed: Complex circuit should have depth 3"
    
    print("✅ All depth calculation tests passed!")

test_depth_calculation()

---

## `count_ops()` - Operation Breakdown

### Signature
```python
QuantumCircuit.count_ops() -> dict[str, int]
```

### Returns
`dict` - Dictionary mapping gate names to their counts

---

In [ ]:
# ============================================================
# count_ops() - BASIC USAGE
# ============================================================

qc = QuantumCircuit(3, 3)
qc.h([0, 1, 2])       # 3 H gates
qc.cx(0, 1)           # 1 CX gate
qc.cx(1, 2)           # 1 CX gate
qc.x(0)               # 1 X gate
qc.z(1)               # 1 Z gate
qc.measure([0, 1, 2], [0, 1, 2])  # 3 measure operations

print("Circuit:")
print(qc.draw())

ops = qc.count_ops()
print(f"\ncount_ops() = {ops}")

print("\nBreakdown:")
for gate, count in ops.items():
    print(f"  {gate}: {count}")

# Verify size matches sum of ops
total = sum(ops.values())
size = qc.size()
assert total == size, f"Sum of ops ({total}) should equal size ({size})"
print(f"\n✅ Verified: sum(count_ops()) = {total} = size() = {size}")

---

## 🏋️ Code Challenges

> These are CODING challenges - solve them by writing code.
> For conceptual questions, see [README.md](./README.md) practice section.

---

In [ ]:
# ============================================================
# CHALLENGE 1: Depth Prediction
# ============================================================
# Task: Create a circuit with EXACTLY depth=2 and size=5
# Hint: Use parallel gates wisely
# ============================================================

def challenge_1():
    """
    Create a circuit with:
    - depth() == 2
    - size() == 5
    
    Returns:
        QuantumCircuit with the required properties
    """
    # YOUR CODE HERE
    qc = QuantumCircuit(3)
    # TODO: Add gates to achieve depth=2, size=5
    
    return qc

# Test your solution
qc = challenge_1()
print("Your circuit:")
print(qc.draw())
print(f"\nDepth: {qc.depth()} (expected: 2)")
print(f"Size: {qc.size()} (expected: 5)")

# Verification
if qc.depth() == 2 and qc.size() == 5:
    print("\n✅ Challenge 1 PASSED!")
else:
    print("\n❌ Challenge 1 FAILED - try again")

In [ ]:
# ============================================================
# SOLUTION: Challenge 1 (Run only after attempting!)
# ============================================================

def challenge_1_solution():
    """Reference solution for Challenge 1."""
    qc = QuantumCircuit(3)
    # Layer 1: 3 parallel H gates (depth +1, size +3)
    qc.h(0)
    qc.h(1)
    qc.h(2)
    # Layer 2: 2 parallel CX gates (depth +1, size +2)
    qc.cx(0, 1)  # Uses q0, q1
    qc.cx(2, 0)  # Uses q2, q0 - BUT wait, q0 is used above!
    # Actually need different approach:
    return qc

# Better solution
def challenge_1_solution_v2():
    """Better solution: Layer 1 has 3 gates, Layer 2 has 2 gates."""
    qc = QuantumCircuit(4)  # Need 4 qubits for true parallel
    # Layer 1: 3 H gates
    qc.h(0)
    qc.h(1)
    qc.h(2)
    # Layer 2: 2 CX gates (parallel)
    qc.cx(0, 1)  # q0, q1
    qc.cx(2, 3)  # q2, q3 (parallel!)
    return qc

qc_sol = challenge_1_solution_v2()
print("Solution circuit:")
print(qc_sol.draw())
print(f"\n✅ depth={qc_sol.depth()}, size={qc_sol.size()}")

In [ ]:
# ============================================================
# CHALLENGE 2: Width Calculation
# ============================================================
# Task: Create a circuit where width() returns 10
# Constraint: Use exactly 6 qubits
# ============================================================

def challenge_2():
    """
    Create a circuit with:
    - num_qubits == 6
    - width() == 10
    
    Returns:
        QuantumCircuit with the required properties
    """
    # YOUR CODE HERE
    # Hint: width = qubits + classical bits
    qc = QuantumCircuit(6, ???)  # What goes in ???
    
    return qc

# Uncomment to test:
# qc = challenge_2()
# print(f"num_qubits: {qc.num_qubits}")
# print(f"width(): {qc.width()}")
# if qc.num_qubits == 6 and qc.width() == 10:
#     print("✅ Challenge 2 PASSED!")
# else:
#     print("❌ Challenge 2 FAILED")

In [ ]:
# ============================================================
# SOLUTION: Challenge 2
# ============================================================

def challenge_2_solution():
    """
    width = num_qubits + num_clbits
    10 = 6 + 4
    """
    qc = QuantumCircuit(6, 4)  # 6 qubits + 4 classical = 10 width
    return qc

qc_sol = challenge_2_solution()
print(f"num_qubits: {qc_sol.num_qubits}")
print(f"num_clbits: {qc_sol.num_clbits}")
print(f"width(): {qc_sol.width()}")
print(f"\n✅ 6 + 4 = 10 width")

---

## Circuit Utilities: copy(), data, global_phase

These utility methods are occasionally tested on the exam.

In [ ]:
# ============================================================
# copy() - Create Independent Copy
# ============================================================

qc_original = QuantumCircuit(2)
qc_original.h(0)
qc_original.cx(0, 1)

# Create copy
qc_copy = qc_original.copy()

# Modify copy
qc_copy.x(0)

print("Original circuit:")
print(qc_original.draw())
print(f"Size: {qc_original.size()}")

print("\nCopied circuit (with extra X):")
print(qc_copy.draw())
print(f"Size: {qc_copy.size()}")

# Verification: original unchanged
assert qc_original.size() == 2, "Original should have 2 gates"
assert qc_copy.size() == 3, "Copy should have 3 gates"
print("\n✅ copy() creates INDEPENDENT circuit - original unchanged")

In [ ]:
# ============================================================
# data Attribute - Access Circuit Instructions
# ============================================================

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.z(1)

print("Circuit:")
print(qc.draw())

# Access instruction data
print("\nqc.data - List of CircuitInstructions:")
for i, instruction in enumerate(qc.data):
    print(f"  {i}: {instruction.operation.name} on qubits {[q._index for q in instruction.qubits]}")

# Get specific instruction
first_gate = qc.data[0].operation
print(f"\nFirst gate: {first_gate.name}")
print(f"First gate qubits: {[q._index for q in qc.data[0].qubits]}")

# Count operations (alternative to count_ops())
gate_names = [inst.operation.name for inst in qc.data]
print(f"\nAll gates: {gate_names}")

print("\n✅ qc.data gives low-level access to all instructions")

In [ ]:
# ============================================================
# global_phase - Circuit Global Phase
# ============================================================

qc = QuantumCircuit(1)
qc.x(0)
qc.x(0)  # X·X = I but may have global phase

print("Circuit with X·X (should be identity):")
print(qc.draw())
print(f"Global phase: {qc.global_phase}")

# Add explicit global phase
qc_with_phase = QuantumCircuit(1)
qc_with_phase.global_phase = np.pi / 4

print("\nCircuit with global phase π/4:")
print(qc_with_phase.draw())
print(f"Global phase: {qc_with_phase.global_phase:.4f} radians")

# Verification
sv1 = Statevector.from_instruction(qc)
print(f"\nState after X·X: {sv1}")

print("\n✅ global_phase: Often appears after optimization/transpilation")
print("📌 EXAM: Global phase is UNOBSERVABLE but affects matrix form")

---

## 📚 Resources

### Code References
- [Qiskit QuantumCircuit API](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit)
- [Qiskit Circuit Library](https://docs.quantum.ibm.com/api/qiskit/circuit_library)

### Related Materials
- [README.md](./README.md) - Conceptual explanations, traps, mnemonics
- [Practice Questions](./README.md#practice-exam) - Text-based exam practice

### Next Notebook
- [Circuit Composition](./circuit_composition.ipynb) - compose(), append(), tensor()

---

### Quick Reference Card

```
┌─────────────────────────────────────────────────────────────────┐
│ CIRCUIT BASICS - QUICK REFERENCE                                │
├─────────────────────────────────────────────────────────────────┤
│ CREATION:                                                       │
│   QuantumCircuit(n, m)          - n qubits, m classical bits   │
│   QuantumCircuit(qr, cr)        - Using named registers        │
├─────────────────────────────────────────────────────────────────┤
│ PROPERTIES (no parentheses):                                    │
│   qc.num_qubits                 - Number of qubits             │
│   qc.num_clbits                 - Number of classical bits     │
├─────────────────────────────────────────────────────────────────┤
│ METHODS (with parentheses):                                     │
│   qc.depth()                    - Critical path length         │
│   qc.size()                     - Total gate count             │
│   qc.width()                    - qubits + clbits              │
│   qc.count_ops()                - Dict of {gate: count}        │
├─────────────────────────────────────────────────────────────────┤
│ KEY FORMULAS:                                                   │
│   width = num_qubits + num_clbits                              │
│   size = sum(count_ops().values())                             │
│   depth = layers (parallel gates = same layer)                 │
└─────────────────────────────────────────────────────────────────┘
```

---

**Notebook verified with Qiskit 1.x** | Last updated: 2025